Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [17]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [18]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [19]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [20]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [32]:
import tensorflow as tf

batch_size = 128
image_size = 28
num_labels = 10

graph = tf.Graph()

num_hidden = 1024

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases_1 = tf.Variable(tf.zeros([num_hidden]))
  
  # Training computation.
  hidden_1 = tf.matmul(tf_dataset, weights_1) + biases_1

  relu = tf.nn.relu(hidden_1)
    
  # Variables.
  weights_2 = tf.Variable(
    tf.truncated_normal([num_hidden, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(relu, weights_2) + biases_2

  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.AdadeltaOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [33]:
with graph.as_default():
    loss = (tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits))
        + (0.01 * (tf.nn.l2_loss(weights_1)
                  + tf.nn.l2_loss(biases_1)
                  + tf.nn.l2_loss(weights_2)
                  + tf.nn.l2_loss(biases_2))))


In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        session.run(prediction, feed_dict={tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(
      session.run(prediction, feed_dict={tf_dataset: test_dataset}),
      test_labels))

Initialized
Minibatch loss at step 0: 3445.638672
Minibatch accuracy: 8.6%
Validation accuracy: 9.0%
Minibatch loss at step 500: 3163.969482
Minibatch accuracy: 71.9%
Validation accuracy: 71.3%
Minibatch loss at step 1000: 3176.345215
Minibatch accuracy: 74.2%
Validation accuracy: 75.3%
Minibatch loss at step 1500: 3150.068115
Minibatch accuracy: 83.6%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 3154.066406
Minibatch accuracy: 82.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 3150.784668
Minibatch accuracy: 82.0%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 3149.597412
Minibatch accuracy: 78.9%
Validation accuracy: 79.2%
Test accuracy: 86.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [35]:
num_steps = 3001

small_train_dataset = train_dataset[0:256, :]
small_train_labels = train_labels[0:256, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        session.run(prediction, feed_dict={tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(
      session.run(prediction, feed_dict={tf_dataset: test_dataset}),
      test_labels))

Initialized
Minibatch loss at step 0: 3497.681396
Minibatch accuracy: 8.6%
Validation accuracy: 7.3%
Minibatch loss at step 500: 3142.918457
Minibatch accuracy: 100.0%
Validation accuracy: 23.5%
Minibatch loss at step 1000: 3142.917725
Minibatch accuracy: 100.0%
Validation accuracy: 23.6%
Minibatch loss at step 1500: 3142.917236
Minibatch accuracy: 100.0%
Validation accuracy: 23.6%
Minibatch loss at step 2000: 3142.917236
Minibatch accuracy: 100.0%
Validation accuracy: 23.6%
Minibatch loss at step 2500: 3142.917236
Minibatch accuracy: 100.0%
Validation accuracy: 23.7%
Minibatch loss at step 3000: 3142.917236
Minibatch accuracy: 100.0%
Validation accuracy: 23.7%
Test accuracy: 26.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases_1 = tf.Variable(tf.zeros([num_hidden]))
  
  # Training computation.
  hidden_1 = tf.matmul(tf_dataset, weights_1) + biases_1

  relu = tf.nn.relu(hidden_1)
  relu = tf.nn.dropout(relu, keep_prob)
    
  # Variables.
  weights_2 = tf.Variable(
    tf.truncated_normal([num_hidden, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(relu, weights_2) + biases_2

  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.AdadeltaOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)


In [37]:
num_steps = 3001

small_train_dataset = train_dataset[0:256, :]
small_train_labels = train_labels[0:256, :]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        session.run(prediction, feed_dict={tf_dataset: valid_dataset, keep_prob : 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(
      session.run(prediction, feed_dict={tf_dataset: test_dataset, keep_prob : 1.0}),
      test_labels))

Initialized
Minibatch loss at step 0: 486.647369
Minibatch accuracy: 11.7%
Validation accuracy: 4.7%
Minibatch loss at step 500: 23.057465
Minibatch accuracy: 82.8%
Validation accuracy: 61.5%
Minibatch loss at step 1000: 9.101736
Minibatch accuracy: 96.9%
Validation accuracy: 65.2%
Minibatch loss at step 1500: 4.787258
Minibatch accuracy: 98.4%
Validation accuracy: 66.4%
Minibatch loss at step 2000: 2.035488
Minibatch accuracy: 98.4%
Validation accuracy: 66.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.2%
Minibatch loss at step 3000: 0.586149
Minibatch accuracy: 98.4%
Validation accuracy: 67.2%
Test accuracy: 73.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
num_steps = 65001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        session.run(prediction, feed_dict={tf_dataset: valid_dataset, keep_prob : 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(
      session.run(prediction, feed_dict={tf_dataset: test_dataset, keep_prob : 1.0}),
      test_labels))

Initialized
Minibatch loss at step 0: 529.793213
Minibatch accuracy: 9.4%
Validation accuracy: 11.0%
Minibatch loss at step 500: 88.466080
Minibatch accuracy: 64.1%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 90.203468
Minibatch accuracy: 70.3%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 70.643806
Minibatch accuracy: 71.9%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 49.993103
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 89.594543
Minibatch accuracy: 73.4%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 73.237701
Minibatch accuracy: 75.0%
Validation accuracy: 82.3%
Minibatch loss at step 3500: 59.720356
Minibatch accuracy: 74.2%
Validation accuracy: 82.6%
Minibatch loss at step 4000: 55.701530
Minibatch accuracy: 80.5%
Validation accuracy: 82.8%
Minibatch loss at step 4500: 45.068100
Minibatch accuracy: 76.6%
Validation accuracy: 83.1%
Minibatch loss at step 5000: 50.597542
Minibatch accuracy: 76.6%
Validat